## Data Generation

In order to train a model to classify faults with metrobuses, we must first get our hands on some data. In this notebook we take a small data set, containing only 100 samples, and use [Markovify](https://github.com/jsvine/markovify) to simulate a larger data set, based on the original samples.

We obtained a small data set by asking our colleagues to describe issues they currently have or had in the past, with their vehicles. 

**Note: you do not need to run this notebook in order to execute the remainder of the workshop. This notebook is solely for data generation. We have already generated the data for you, and it can be found in the `/dataset` folder.**

Let's take a look at that data: 

In [ ]:
import pandas as pd

In [ ]:
pd.set_option('display.max_colwidth', None)
df = pd.read_csv('dataset/response.csv') 
df.sample(10)

As you can see, the data contains a range of information including, the time the error was recorded, and the type of issue, as well as a description of the problem. The information we are really interested in is the type of issue, and the description - let's go ahead and process this data, extracting only the info we need!

In [ ]:
df = df.fillna('')
df['response']=df.iloc[:,3]+df.iloc[:,5]+df.iloc[:,6]
df['issue'] = df.iloc[:,1]
df['symptom'] = df.iloc[:,2] + df.iloc[:,4]
subset = df.iloc[:,-3:]
subset.sample(10)

This looks much nicer! Let's explore our data:

In [ ]:
subset.issue.unique()

We only have 3 categories of issue - 'Brakes', 'Starter' and 'Other'. 

How much data do we have?

In [ ]:
subset.shape

In [ ]:
subset['issue'].value_counts() 

Only 109 rows - That's not likely to be enough to train a successful categorization model.

Most of the data falls into the 'other' category.

In the rest of this notebook, we use Markovify to generate more data for each class of issue. Markovify is a Markov Chain generator, and we are going to use it to simulate more responses for each type of issue.

First we install markovify, which is in our `nb-requirements.txt` file:

In [ ]:
!pip install -r nb-requirements.txt

In [ ]:
import markovify
import csv

In [ ]:
def train_markov_type(data, issue):
    return markovify.Text(data[data["issue"] == issue].response, retain_original=False, state_size=2)

#Function takes one of the 'issue' models and creates a randomly-generated sentence of length up to 100 characters.  
def make_sentence(model, length=100):
    return model.make_short_sentence(length, max_overlap_ratio = .7, max_overlap_total=15)

#built models
other_model = train_markov_type(subset, "Other")
brakes_model = train_markov_type(subset, "Brakes")
starter_model = train_markov_type(subset, "Starter")

We can combine these models with relative weights:

In [ ]:
import numpy

def generate_cases(models, weights=None):
    if weights is None:
        weights = [1] * len(models)
    
    choices = [] # Array of tuples of weight and models
    
    total_weight = float(sum(weights))
    
    for i in range(len(weights)):
        choices.append((float(sum(weights[0:i+1])) / total_weight, models[i]))
    
    # Return a tuple of model and category that are randomly selected by given weights.
    def choose_model():
        r = numpy.random.uniform()
        for (model_weight, model) in choices:
            if r <= model_weight:
                return model
        return choices[-1][1]


    while True:
        local_model = choose_model() 
        # local_model[0]) is the markovify model, local_model[1] is the category
        yield make_sentence(local_model[0]), local_model[1]
   


We can now use this code to generate new sentences:

In [ ]:
import numpy as np

generated_cases = generate_cases([(other_model,'other'), (brakes_model,'brakes'), (starter_model,'starter')], [14,7,7])

# Tuples with sentence and category
sentence_tuples = [next(generated_cases)  for i in range(1000)]  # create 200 sentence/category tuples

In [ ]:
sentence_tuples

In [ ]:
len(sentence_tuples)

We can save these new issues and responses to file, and we will use this file later to train our model. 

In [ ]:
# Write to csv file
with open('dataset/testdata1.csv', 'w') as file:
    writer = csv.writer(file, delimiter=',', lineterminator='\n')
    writer.writerows(sentence_tuples)

At this point we have created a new data set, and we can transform this data and train a classification model. 

Let's head to notebook [01-Create-Claims-Classification.ipynb](01-Create-Claims-Classification.ipynb), where we process the data and train a classifier. 